In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark=SparkSession.builder.appName('bench_employee_project').getOrCreate()
df=spark.read.csv("dbfs:/FileStore/bench_employees_costsavings-1.csv",header=True,inferSchema=True)
df.display()

empid,employee_name,years_exp,in_project,project_name,benchcode,benchdate,leave_plan_days,salary_month,certification,certification_completetion_date,certification_completed_or_not
1,EID_22713,13,yes,apac,null,null,null,150000,null,null,null
2,EID_9658,12,no,null,b1001,2023-11-01,5,200000,databricks professional,2023-11-15,yes
3,EID_22203,13,yes,apac,null,null,null,300000,null,null,null
4,EID_7652,10,yes,apac,null,null,null,120000,null,null,null
5,EID_6516,9,No,null,b1001,2023-10-01,0,130000,databricks associate,2023-10-15,no
6,EID_20283,10,yes,apac,null,null,null,150000,null,null,null
7,EID_21014,8,no,null,b1001,2023-10-21,10,200000,databricks associate,2023-11-06,no
8,EID_7693,12,yes,apac,null,null,null,300000,null,null,null
9,EID_13232,18,yes,apac,null,null,null,120000,null,null,null
10,EID_6515,18,No,null,b1001,2023-09-01,0,130000,databricks professional,2023-09-15,yes


In [0]:
from pyspark.sql.functions import current_date, to_date, datediff,col

# Calculate "Daily_Salary" based on "salary_month"
df = df.withColumn("Daily_Salary", col("salary_month") / 30)
df = df.withColumn("benchdate", to_date(col("benchdate")))
df = df.withColumn("days_difference", datediff(current_date(), col("benchdate")))
df = df.withColumn("Bench_salary", col("Daily_Salary") * (col("days_difference") - col("leave_plan_days")))

# Display the DataFrame
df.display()


empid,employee_name,years_exp,in_project,project_name,benchcode,benchdate,leave_plan_days,salary_month,certification,certification_completetion_date,certification_completed_or_not,Daily_Salary,days_difference,Bench_salary
1,EID_22713,13,yes,apac,null,null,null,150000,null,null,null,5000.0,null,null
2,EID_9658,12,no,null,b1001,2023-11-01,5,200000,databricks professional,2023-11-15,yes,6666.666666666667,17,80000.0
3,EID_22203,13,yes,apac,null,null,null,300000,null,null,null,10000.0,null,null
4,EID_7652,10,yes,apac,null,null,null,120000,null,null,null,4000.0,null,null
5,EID_6516,9,No,null,b1001,2023-10-01,0,130000,databricks associate,2023-10-15,no,4333.333333333333,48,208000.0
6,EID_20283,10,yes,apac,null,null,null,150000,null,null,null,5000.0,null,null
7,EID_21014,8,no,null,b1001,2023-10-21,10,200000,databricks associate,2023-11-06,no,6666.666666666667,28,120000.0
8,EID_7693,12,yes,apac,null,null,null,300000,null,null,null,10000.0,null,null
9,EID_13232,18,yes,apac,null,null,null,120000,null,null,null,4000.0,null,null
10,EID_6515,18,No,null,b1001,2023-09-01,0,130000,databricks professional,2023-09-15,yes,4333.333333333333,78,338000.0


In [0]:

# Filter the DataFrame for "bench_salary" greater than 400000
filtered_df = df.filter(col("Bench_salary") > 400000)

filtered_df.display()

empid,employee_name,years_exp,in_project,project_name,benchcode,benchdate,leave_plan_days,salary_month,certification,certification_completetion_date,certification_completed_or_not,Daily_Salary,days_difference,Bench_salary
15,EID_20157,2,No,null,b1001,2023-07-07,30,120000,dp900,2023-07-15,yes,4000.0,134,416000.0
17,EID_20748,1,no,null,b1001,2023-08-15,5,150000,dp900,2023-08-22,no,5000.0,95,450000.0


In [0]:
#email alert in databricks Notebook

import smtplib
from email.mime.text import MIMEText

def send_email(subject, body, to_email, smtp_server, smtp_port, smtp_username, smtp_password):
    # Set up the email content
    message = MIMEText(body)
    message["Subject"] = subject
    message["From"] = smtp_username
    message["To"] = to_email

    # Connect to the SMTP server
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        # Log in to the SMTP server if username and password are provided
        if smtp_username and smtp_password:
            server.login(smtp_username, smtp_password)

        # Send the email
        server.sendmail(smtp_username, to_email, message.as_string())

# Set your email configuration
subject = "Databricks Alert"
body = "This is a test email from Databricks notebook."
to_email = "recipient@example.com"
smtp_server = "your_smtp_server"
smtp_port = 587  # Use the appropriate port for your SMTP server
smtp_username = "your_email@example.com"
smtp_password = "your_email_password"

# Send the email
send_email(subject, body, to_email, smtp_server, smtp_port, smtp_username, smtp_password)
